This notebook explains how to use the diffthermo package to fit OCV models.

First, install the `diffthermo` package according to the README file. 

In [ ]:
conda create -n diffthermo python=3.11  torch=2.0.0 numpy=1.24 matplotlib pandas
conda activate diffthermo 
git clone https://github.com/BattModels/Diffthermo_OCV_paper.git
cd Diffthermo_OCV_paper
pip install .

To fit the OCV model, you need a csv file which contains the OCV VS SoC data. The first column of the file is SoC data, the second column is the OCV value. The OCV VS SoC data may come from experimental results or digitizing the results from figures of existing literature that contains OCV VS SoC information. 

In [2]:
import pandas as pd 
datafile_name = 'graphite.csv' # this is the csv file that you prepard, it should contain OCV VS SoC data
df = pd.read_csv(datafile_name,header=None)
print(df)

           0         1
0   0.020261  0.493823
1   0.021986  0.485258
2   0.022533  0.477210
3   0.023329  0.466564
4   0.024362  0.456673
..       ...       ...
94  0.918431  0.059359
95  0.930202  0.052110
96  0.939606  0.044030
97  0.947238  0.035824
98  0.954031  0.028318

[99 rows x 2 columns]


Now, you simply need to feed the prepared csv file that contains OCV VS SoC data into the diffthermo package. 

Put the name of your csv file as `datafile_name`. `number_of_Omegas` is a hyperparameter that you should choose. I suggest to start with small numbers (3 or 4 as examples), and increase the number if the fitted model does not look good. The `total_training_epochs` specifies how many round of updates the parameters (U0 and Omegas) will have. You should set the number accordingly, so that the loss value is minimized. I recommend the learning_rate to be 1000.0 (since the value of Omegas are usually large), you can leave this value unchanged. `loss_threshold` controls the minimum value of loss, below which the fitting process will automatically stop. 

Now let's just run the model and see what's going to happen:

In [3]:
from diffthermo.utils import train
params_list = train(datafile_name='graphite.csv', 
                    number_of_Omegas=6, 
                    learning_rate = 1000.0, 
                    total_training_epochs = 200,
                    loss_threshold = 0.01)

Epoch   0  Loss 30.0609     Omega0 -1722.7913 Omega1 -1820.9719 Omega2 -640.9189 Omega3 -60.9683 Omega4 -1713.9203 Omega5 -1771.9647 G0 -25058.0020       
Epoch   1  Loss 25.7058     Omega0 -1135.4871 Omega1 -2699.1172 Omega2 -994.1658 Omega3 -993.9672 Omega4 -2469.2134 Omega5 -2735.0867 G0 -24058.4629       
Epoch   2  Loss 21.7826     Omega0 -365.7859 Omega1 -3430.3489 Omega2 -867.2512 Omega3 -1859.1750 Omega4 -2951.1152 Omega5 -3665.3408 G0 -23061.9648       
Epoch   3  Loss 18.2313     Omega0 448.2627 Omega1 -4021.4067 Omega2 -518.5709 Omega3 -2667.2686 Omega4 -3243.5088 Omega5 -4570.2480 G0 -22071.7168       
Epoch   4  Loss 15.0845     Omega0 1195.2362 Omega1 -4526.3154 Omega2 -81.1008 Omega3 -3443.2788 Omega4 -3465.2881 Omega5 -5454.7529 G0 -21091.5605       
Epoch   5  Loss 12.4089     Omega0 1707.0084 Omega1 -4953.3569 Omega2 284.3814 Omega3 -4195.6572 Omega4 -3711.3989 Omega5 -6331.3589 G0 -20125.7051       
Epoch   6  Loss 10.1198     Omega0 1909.2925 Omega1 -5311.5781 Omega

The loss value is decreasing in each epoch. Since we set the `total_training_epochs` to be 200, the fitting process stops at 200 epochs. You can adjust this number if you want to further minimize the loss value in order to get a more accurate OCV model.

Now, we have succesfully fitted an OCV model for graphite. To access your fitted model, call `write_ocv_functions` function:

In [4]:
from diffthermo.utils import write_ocv_functions
write_ocv_functions(params_list)

Found 2 phase coexistence region(s):
From x=0.1569776684045792 to x=0.3498985171318054
From x=0.5453399419784546 to x=0.8350558280944824





 Fitting Complete.

Fitted OCV function written in PyBaMM function (copy and paste readay!):

###################################

import numpy as np
import pybamm
from pybamm import exp, log, tanh, constants, Parameter, ParameterValues

def fitted_OCP(sto):
    _eps = 1e-7
    # rk params
    G0 = -11686.997070 # G0 is the pure substance gibbs free energy 
    Omega0 = -2697.453369 
    Omega1 = -5728.828125 
    Omega2 = 4744.390137 
    Omega3 = 7466.139160 
    Omega4 = -19736.783203 
    Omega5 = -20078.644531 
    Omegas =[Omega0, Omega1, Omega2, Omega3, Omega4, Omega5]
    # phase boundary 0
    x_alpha_0 = 0.1569776684045792
    x_beta_0 = 0.3498985171318054
    mu_coex_0 = -12697.3496093750000000
    is_outside_miscibility_gap_0 = (sto<x_alpha_0) + (sto>x_beta_0)
    # phase boundary 1
    x_alpha_1 = 0.5453399419784546
    x_beta_1 = 0.

You can directly copy & paste the fitted pybamm OCV function as printed in the terminal, or you can find the function in `fitted_ocv_functions.py`  file under the same directory where you put your csv file in. 